### Dataset Ingestion

Dataset from: https://www.kaggle.com/nicapotato/womens-ecommerce-clothing-reviews

In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"  
os.environ["CUDA_VISIBLE_DEVICES"]="1"

import random
import numpy as np
import pandas as pd
from sklearn import model_selection

import torch
from torch import nn
from torch.utils import data

from transformers import (WEIGHTS_NAME, 
                          BertConfig, BertForSequenceClassification, BertTokenizer,
                          RobertaConfig, RobertaForSequenceClassification, RobertaTokenizer,
                          XLMConfig, XLMForSequenceClassification, XLMTokenizer, 
                          XLNetConfig, XLNetForSequenceClassification, XLNetTokenizer,
                          DistilBertConfig, DistilBertForSequenceClassification, DistilBertTokenizer)
from transformers import AdamW, WarmupLinearSchedule
from transformers import AutoModelForSequenceClassification, AutoTokenizer

In [2]:
df = pd.read_csv("../input/ClothingReviews.csv")
df.head()

,id,text,recommended
0,0,Absolutely wonderful - silky and sexy and comf...,1
1,1,Love this dress! it's sooo pretty. i happene...,1
2,2,I had such high hopes for this dress and reall...,0
3,3,"I love, love, love this jumpsuit. it's fun, fl...",1
4,4,This shirt is very flattering to all due to th...,1


In [3]:
df["recommended"].value_counts()

1    19314
0     4172
Name: recommended, dtype: int64

In [4]:
train_df, test_df = model_selection.train_test_split(df, test_size=0.3, random_state=2019)
print("Train data shape is : ",train_df.shape)
print("Test data shape is : ",test_df.shape)

Train data shape is :  (16440, 3)
Test data shape is :  (7046, 3)


### Custom Functions

#### Conversion of text to features for Transformer models

In [6]:
def convert_text_to_features(examples, tokenizer,
                                      max_length=512,
                                      pad_on_left=False,
                                      pad_token=0,
                                      pad_token_segment_id=0,
                                      mask_padding_with_zero=True):
    """
    Loads a data file into a list of ``InputFeatures``
    Args:
        examples: List of ``InputExamples`` or ``tf.data.Dataset`` containing the examples.
        tokenizer: Instance of a tokenizer that will tokenize the examples
        max_length: Maximum example length
        pad_on_left: If set to ``True``, the examples will be padded on the left rather than on the right (default)
        pad_token: Padding token
        pad_token_segment_id: The segment ID for the padding token (It is usually 0, but can vary such as for XLNet where it is 4)
        mask_padding_with_zero: If set to ``True``, the attention mask will be filled by ``1`` for actual values
            and by ``0`` for padded values. If set to ``False``, inverts it (``1`` for padded values, ``0`` for
            actual values)
    Returns:
        If the ``examples`` input is a ``tf.data.Dataset``, will return a ``tf.data.Dataset``
        containing the task-specific features. If the input is a list of ``InputExamples``, will return
        a list of task-specific ``InputFeatures`` which can be fed to the model.
    """
    features = [[],[],[]]
    for (ex_index, example) in enumerate(examples):
        if ex_index % 10000 == 0:
            print("Writing example %d" % (ex_index))

        inputs = tokenizer.encode_plus(
            example,
            None,
            add_special_tokens=True,
            max_length=max_length,
        )
        input_ids, token_type_ids = inputs["input_ids"], inputs["token_type_ids"]

        # The mask has 1 for real tokens and 0 for padding tokens. Only real
        # tokens are attended to.
        attention_mask = [1 if mask_padding_with_zero else 0] * len(input_ids)

        # Zero-pad up to the sequence length.
        padding_length = max_length - len(input_ids)
        if pad_on_left:
            input_ids = ([pad_token] * padding_length) + input_ids
            attention_mask = ([0 if mask_padding_with_zero else 1] * padding_length) + attention_mask
            token_type_ids = ([pad_token_segment_id] * padding_length) + token_type_ids
        else:
            input_ids = input_ids + ([pad_token] * padding_length)
            attention_mask = attention_mask + ([0 if mask_padding_with_zero else 1] * padding_length)
            token_type_ids = token_type_ids + ([pad_token_segment_id] * padding_length)

        assert len(input_ids) == max_length, "Error with input length {} vs {}".format(len(input_ids), max_length)
        assert len(attention_mask) == max_length, "Error with input length {} vs {}".format(len(attention_mask), max_length)
        assert len(token_type_ids) == max_length, "Error with input length {} vs {}".format(len(token_type_ids), max_length)

        if ex_index < 1:
            print("*** Example ***")
            print("input_ids: %s" % " ".join([str(x) for x in input_ids]))
            print("attention_mask: %s" % " ".join([str(x) for x in attention_mask]))
            print("token_type_ids: %s" % " ".join([str(x) for x in token_type_ids]))

        features[0].append(input_ids)
        features[1].append(attention_mask)
        features[2].append(token_type_ids)

    return features

In [7]:
def seed_everything(seed=123):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.backends.cudnn.deterministic = True

In [8]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [9]:
MODEL_CLASSES = {
    'bert': (BertConfig, BertForSequenceClassification, BertTokenizer),
    'xlnet': (XLNetConfig, XLNetForSequenceClassification, XLNetTokenizer),
    'xlm': (XLMConfig, XLMForSequenceClassification, XLMTokenizer),
    'roberta': (RobertaConfig, RobertaForSequenceClassification, RobertaTokenizer),
    'distilbert': (DistilBertConfig, DistilBertForSequenceClassification, DistilBertTokenizer)
}

#### Initializing the BERT Model 

In [10]:
model_name = "bert"
pretrained_model_name = "bert-base-uncased"
n_classes = 1

config_class, model_class, tokenizer_class = MODEL_CLASSES[model_name]
config = config_class.from_pretrained(pretrained_model_name)
tokenizer = tokenizer_class.from_pretrained(pretrained_model_name, do_lower_case=True)
model = model_class.from_pretrained(pretrained_model_name, num_labels=1)

I1111 21:18:36.259483 140512594204416 configuration_utils.py:151] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /home/srk/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.bf3b9ea126d8c0001ee8a1e8b92229871d06d36d8808208cc2449280da87785c
I1111 21:18:36.262970 140512594204416 configuration_utils.py:168] Model config {
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 2,
  "output_attentions": false,
  "output_hidden_states": false,
  "output_past": true,
  "pruned_heads": {},
  "torchscript": false,
  "type_vocab_size": 2,
  "use_bfloat16": false,
  "vocab_size": 30522
}

I1111 21:18:36.574251 

#### Converting the data into required format

In [11]:
max_length = 128
train_df["text"] = train_df["text"].astype(str).fillna("NA")
train_features = convert_text_to_features(train_df["text"], tokenizer, max_length=max_length)

/home/srk/env/DS2/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


Writing example 0
*** Example ***
input_ids: 101 2009 2003 2061 3722 2021 11552 1998 3376 999 1045 2572 1019 1005 1017 1998 6036 6053 1998 4469 2235 16142 2033 6669 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
Writing example 10000


In [12]:
X = torch.tensor(train_features[0], dtype=torch.long)
X_mask = torch.tensor(train_features[1], dtype=torch.long)
X_seg_ids = torch.tensor(train_features[2], dtype=torch.long)
y = train_df["recommended"].values
y = torch.tensor(y[:,np.newaxis], dtype=torch.float32)

batch_size = 8
train_dataset = data.TensorDataset(X, X_mask, X_seg_ids, y)
train_loader = data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

#### Model Parameters

In [13]:
n_epochs = 1
accumulation_steps = 1

param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]
num_train_optimization_steps = int(n_epochs*len(train_dataset)/batch_size/accumulation_steps)
num_warmup_steps = int(0.05*num_train_optimization_steps)
optimizer = AdamW(optimizer_grouped_parameters, lr=2e-5, correct_bias=False)
scheduler = WarmupLinearSchedule(optimizer,
                                 warmup_steps=num_warmup_steps,
                                 t_total=num_train_optimization_steps)

#### Model Training

In [15]:
seed_everything()
model.to(device)
loss_fn = nn.BCEWithLogitsLoss()

for epoch in range(n_epochs):
    model.train()
    for x_batch, x_mask, x_seg_ids, y_batch in train_loader:
        outputs = model(x_batch.to(device),
                        attention_mask=x_mask.to(device),
                        token_type_ids=x_seg_ids.to(device),
                        labels=None)
        y_pred = outputs[0]
        loss = loss_fn(y_pred, y_batch.to(device))
        loss.backward()
        scheduler.step()
        optimizer.step()
        optimizer.zero_grad()

#### Preparation of test dataset

In [16]:
test_df["text"] = test_df["text"].astype(str).fillna("NA")
test_features = convert_text_to_features(test_df["text"], tokenizer, max_length=max_length)

test_X = torch.tensor(test_features[0], dtype=torch.long)
test_X_mask = torch.tensor(test_features[1], dtype=torch.long)
test_X_seg_ids = torch.tensor(test_features[2], dtype=torch.long)
test_y = test_df["recommended"].values

test_dataset = data.TensorDataset(test_X, test_X_mask, test_X_seg_ids)
test_loader = data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

/home/srk/env/DS2/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


Writing example 0
*** Example ***
input_ids: 101 2023 2003 1037 2307 4377 2005 2621 1998 2200 4257 17989 999 1045 2572 1019 1005 1018 1998 2071 2031 2766 2125 1996 3180 3091 2021 4900 2000 2175 2007 1996 20146 2061 2008 1045 2071 4929 14201 1012 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


In [17]:
preds = np.zeros([len(test_dataset), 1])
model.eval()
for i, (x_batch, x_mask, x_seg_ids) in enumerate(test_loader):
    outputs = model(x_batch.to(device),
                    attention_mask=x_mask.to(device),
                    token_type_ids=x_seg_ids.to(device),
                    labels=None)
    y_pred = sigmoid(outputs[0].detach().cpu().numpy())
    preds[i*batch_size:(i+1)*batch_size, :] = y_pred
    
from sklearn import metrics
metrics.roc_auc_score(test_y, preds)

0.9629027490393143

#### Parameters to Optimize

* Max Sequence Length
* Batch size
* Learning rate
* Number of epochs
* Gradient accumulation steps
* Warmup steps